In [1]:
# %matplotlib widget
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.io import savemat

from scikitMedicalRobot.ros import tf2_helper, ros_helper
import scikitMedicalRobot.utilities as tools

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

Definition: rosbag files

In [2]:
bag_file = Path('test_data/rosbag_data/20230112_145234_743087_endmill_rotV0.8_D3.0_d0.5_cart3_theta0_exp0.bag')
saving_path = bag_file.parent / Path(bag_file.stem)
tools.ensure_dir(saving_path)

Definition: topics for saving

```
topics_for_saving = {
    topic1_name:[[link_name, link_name], [link_name, link_name]],
    topic2_name:[],
    ...
}
```

In [3]:
topics_for_saving = {
    '/ft_sensor_filtered': [['base_link', 'tool0'], ['base_link', 'tool_link'], ['tool0', 'tool_link']], 
    '/polaris_ros_node/targets': [], 
    '/servo_drill/AD7606_AD_Value': [],
}

Helper Functions

In [4]:
def save_df_as_pickle(df: pd.DataFrame, pickle_fn: Path):
    df.to_pickle(pickle_fn)

def save_df_as_mat(df: pd.DataFrame, mat_fn: Path):
    mdic = {}
    for key in df.columns:
        mdic[key] = df[key].values
    savemat(mat_fn, mdic)

Load Rosbag File

In [5]:
bag_extractor = ros_helper.rosbag_extractor(str(bag_file))
print(f"{bag_file.stem} - {bag_extractor.file_size(human=True)}")

20230112_145234_743087_endmill_rotV0.8_D3.0_d0.5_cart3_theta0_exp0 - 17.31 MB


Extract Data

In [6]:
data_dict = bag_extractor.extract_data_from_topics(topics_for_saving, with_tqdm=True)
print(data_dict.keys())

  0%|          | 0/84729 [00:00<?, ?it/s]

dict_keys(['/ft_sensor_filtered', '/polaris_ros_node/targets', '/servo_drill/AD7606_AD_Value'])


## Extract Data from Rosbag

Without tf data

In [7]:
topic = '/servo_drill/AD7606_AD_Value'

In [8]:
n_frames = len(data_dict[topic]['timestamp'])
ad7606_arr = np.zeros((n_frames, 9))

for idx in range(n_frames):
    msg = data_dict[topic]['msg'][idx]
    ad_value = msg.ad_value
    ad7606_arr[idx, 0] = msg.header.stamp.to_sec()
    ad7606_arr[idx, 1:] = ad_value

In [9]:
df_ad7606 = pd.DataFrame(
    columns=['timestamp', 'ch0', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7'], 
    data=ad7606_arr
)

save_df_as_pickle(df_ad7606, saving_path / Path('df_ad7606.pkl'))
save_df_as_mat(df_ad7606, saving_path / Path('df_ad7606.mat'))

In [10]:
data = pd.read_pickle(saving_path / Path('df_ad7606.pkl'))
data

,timestamp,ch0,ch1,ch2,ch3,ch4,ch5,ch6,ch7
0,1.673506e+09,0.005341,0.005493,0.004578,1.767731,1.772919,1.768951,1.773376,1.768494
1,1.673506e+09,0.005493,0.005646,0.004578,1.768036,1.773224,1.769257,1.773682,1.768951
2,1.673506e+09,0.005798,0.005798,0.004730,1.768341,1.773529,1.769562,1.773987,1.769104
3,1.673506e+09,0.005798,0.005646,0.004730,1.768341,1.773376,1.769562,1.773987,1.768951
4,1.673506e+09,0.005798,0.005646,0.004730,1.768188,1.773224,1.769257,1.773834,1.768951
...,...,...,...,...,...,...,...,...,...
41520,1.673506e+09,0.005646,0.005798,0.005035,1.768188,1.773224,1.769409,1.773834,1.768799
41521,1.673506e+09,0.005798,0.005798,0.004883,1.768188,1.773376,1.769257,1.773834,1.768799
41522,1.673506e+09,0.005798,0.005951,0.005035,1.768188,1.773376,1.769409,1.773834,1.768951
41523,1.673506e+09,0.005493,0.005493,0.004730,1.768341,1.773376,1.769409,1.773834,1.768951


With tf data

In [11]:
topic = '/ft_sensor_filtered'

In [12]:
n_frames = len(data_dict[topic]['timestamp'])
ft_val_arr = np.zeros((n_frames, 1+6))

tf_names = []
for k in data_dict[topic].keys():
    if '->' in k:
        tf_names.append(k)
tf_arr = np.zeros((n_frames, 1+7*len(tf_names)))

for idx in range(n_frames):
    msg = data_dict[topic]['msg'][idx]
    tf_arr[idx, 0] = msg.header.stamp.to_sec()
    # tf
    for j, k in enumerate(tf_names):
        (position, quat) = data_dict[topic][k][idx]
        if position is not None and quat is not None:
            tf_arr[idx, (1+7*j):(1+7*(j+1))] = position + quat
        else:
            tf_arr[idx, (1+7*j):(1+7*(j+1))] = None
    # force
    ft_val = [
        msg.wrench.force.x, msg.wrench.force.y, msg.wrench.force.z, 
        msg.wrench.torque.x, msg.wrench.torque.y, msg.wrench.torque.z]
    ft_val_arr[idx, 0] = msg.header.stamp.to_sec()
    ft_val_arr[idx, 1:] = np.array(ft_val)

In [13]:
df_force_tool0 = pd.DataFrame(
    columns=['timestamp', 'fx', 'fy', 'fz', 'tx', 'ty', 'tz'], 
    data=ft_val_arr
)
columns=['timestamp']
for t_n in tf_names:
    for c in ['x', 'y', 'z', 'qx', 'qy', 'qz', 'qw']:
        columns.append(f'{t_n}_{c}')
        
df_tf = pd.DataFrame(
    columns=columns,
    data=tf_arr
)

save_df_as_pickle(df_force_tool0, saving_path / Path('df_force_tool0.pkl'))
save_df_as_mat(df_force_tool0, saving_path / Path('df_force_tool0.mat'))
save_df_as_pickle(df_tf, saving_path / Path('df_tf.pkl'))
save_df_as_mat(df_tf, saving_path / Path('df_tf.mat'))

In [14]:
data = pd.read_pickle(saving_path / Path('df_force_tool0.pkl'))
data

,timestamp,fx,fy,fz,tx,ty,tz
0,1.673506e+09,5.040294,-1.846640,-3.551726,0.493764,-0.070779,0.008543
1,1.673506e+09,5.038367,-1.836571,-3.556430,0.493896,-0.070633,0.008745
2,1.673506e+09,5.045437,-1.827355,-3.554467,0.494653,-0.070680,0.008675
3,1.673506e+09,5.049208,-1.836391,-3.537105,0.495210,-0.070637,0.008723
4,1.673506e+09,5.045622,-1.835961,-3.537261,0.495360,-0.070582,0.008619
...,...,...,...,...,...,...,...
40705,1.673506e+09,4.970628,-1.857437,-3.538025,0.495441,-0.070871,0.009002
40706,1.673506e+09,4.964418,-1.864569,-3.552920,0.494940,-0.070768,0.008858
40707,1.673506e+09,4.965260,-1.866684,-3.558738,0.494372,-0.070814,0.008875
40708,1.673506e+09,4.955685,-1.855092,-3.568309,0.493958,-0.070882,0.008955
